In [ ]:
import numpy as np
import os
import sys
import pandas as pd

import sklearn as sk
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns

import pylab as pl
import h5py

import tensorflow as tf

os.environ['TF_CPP_MIN_LOG_LEVEL']='2'
os.environ['CUDA_VISIBLE_DEVICES']="0" 

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)


from sklearn.metrics import confusion_matrix


base_dir = '/media/tord/T7/Thesis_ssd/MasterThesis3'
os.chdir(base_dir)
from Classes.DataProcessing.LoadData import LoadData
from Classes.DataProcessing.HelperFunctions import HelperFunctions
from Classes.DataProcessing.DataHandler import DataHandler
from Classes.DataProcessing.TimeAugmentor import TimeAugmentor
from Classes.DataProcessing.NoiseAugmentor import NoiseAugmentor
from Classes.DataProcessing.RamLoader import RamLoader
from Classes.DataProcessing.RamGenerator import RamGenerator
from Classes.Modeling.InceptionTimeModel import InceptionTimeModel
from Classes.Modeling.NarrowSearchIncepTime import NarrowSearchIncepTime
from Classes.Modeling.GridSearchResultProcessor import GridSearchResultProcessor
from Classes.Modeling.CustomCallback import CustomCallback
from Classes.Modeling.ResultFitter import ResultFitter
from Classes.Scaling.ScalerFitter import ScalerFitter
from Classes.Scaling.MinMaxScalerFitter import MinMaxScalerFitter
from Classes.Scaling.StandardScalerFitter import StandardScalerFitter
import json
#from Classes import Tf_shutup
#Tf_shutup.Tf_shutup()

helper = HelperFunctions()

import sys
ISCOLAB = 'google.colab' in sys.modules

import random
import pprint

In [1]:
load_args = {
    'earth_explo_only' : False,
    'noise_earth_only' : False,
    'noise_not_noise' : True,
    'downsample' : True,
    'upsample' : True,
    'frac_diff' : 1,
    'seed' : 1,
    'subsample_size' : 0.1,
    'balance_non_train_set' : True,
    'use_true_test_set' : False,
    'even_balance' : True
}
loadData = LoadData(**load_args)
full_ds, train_ds, val_ds, test_ds = loadData.get_datasets()
noise_ds = loadData.noise_ds
handler = DataHandler(loadData)

NameError: name 'LoadData' is not defined

In [16]:
class NarrowOptimizer(GridSearchResultProcessor):

    """
    This class functions as an heuristic that will attempt to reach a local minima. The class can either start off an existing search, or can start its own. The process looks a little like this:
    1. Select the best model from existing result file (if using a file)
    2. Use the best model / start model as the foundation. Create a search space around this in terms of hyperparameters.
    3. Do a narrow search on the generated search space. If quick_mode = True, then if a better model is found during the narrow search, replace the base model with this and return to step 2. 
    4. Repeat steps 1-3
    

    Notes:
    
    - Would like this to be as robust as possible, and not dependent on InceptionTime. Want to be able to use this class for any model really.
        - This can be challenging when creating dictionaries, as annoyingly, the models use 2 seperate dictionaries for initilization.
    
    Drawbacks, potential points of failure:
     - The filtering method is very simple, and assumes that less than half of the good models are buggy. This is definitely not necessarily the case, and will cause this class to potentially try to optimize a lost cause. HOW CAN THIS BE SOLVED!?!?!?!? BY FIXING THE ORIGINAL BUG YOU DUMB FUCK
     - The way results are processed, requires a model_grid and a hyper_grid. This design choice is the root of sooo many problems, and may lead to a less than robust implementation of this class. This can lead to different versions. Potential soution: Use this class as a parent class, and have children objects that are specialized for each type of model. 

    
    """

    def __init__(self, depth, quick_mode = False, continue_from_result_file = False, result_file_name = "", start_grid = []):
        self.depth = depth
        self.quick_mode = quick_mode
        self.continue_from_result_file = continue_from_result_file
        self.result_file_name = result_file_name
        self.start_grid = start_grid

    def run(self, result_file_name, num_classes, optimize_metric = ['val_accuracy', 'val_f1'], nr_candidates = 10):
        """
        Self explanatory

        PARAMS:
        --------------
        result_file_name: (str) Name of the file to be used. If continue_from_result == False, then this will not be used
        num_classes: (int)
        optimize_metric: [string, string] Optimization criteria. First element will be most significant.
        nr_candidates: (int) Number of model candidates that will be considered in the first step sort.

        """
        if self.quick_mode:
            if self.continue_from_result_file:
                print(f"Quick mode, starting of result file: {result_file_name}")
                best_model = self.get_best_model(result_file_name, num_classes, optimize_metric, nr_candidates)

                best_model_dict = self.delete_metrics(best_model).iloc[0].to_dict()
                best_model_dict = self.adapt_best_model_dict(best_model_dict)
                return best_model_dict

            else:
                raise Exception("Not continuing training from result file is not yet implemented. Suspected to be unused.")
        else:
            if self.continue_from_result_file:
                print(f"Exhaustive mode, starting of result file: {result_file_name}")
            else:
                raise Exception("Not continuing training from result file is not yet implemented. Suspected to be unused.")
        return

    
    def quick_mode(self, result_file_name, num_classes, optimize_metric):
        pass

    def get_best_model(self, result_file_name, num_classes, optimize_metric, nr_candidates):
        # Clear nan values
        self.clear_nans(result_file_name, num_classes)
        results_df = self.get_results_df_by_name(result_file_name, num_classes)
        df_f1 = results_df.copy()
        # Add f1 stats
        df_f1 = self.add_f1_stats(df_f1)
        # Sort by sort conditions
        sorted_df = self.sort_df(df_f1, optimize_metric)
        # Get the top nr_candidates
        best_initial_candidates = sorted_df.copy().head(nr_candidates)
        # Attempt to only select models which have the best f1 score, and first part of the sort conditions
        # This is due to (likely) bug that has some models perform really well in one metric, but terrible in other metrics. The working assumption is that models with high f1, are good.
        # TODO: Consider just switching the optimizer metrics here. Without the current BUG with strange training metrics (and inconsistent metrics wrt. the confusion matrix) this is a good opportunity to optimize with two metrics.
        best_initial_sorted_by_f1 = self.sort_df(best_initial_candidates, ['val_f1', optimize_metric[0]])
        # Select nr_candidates//2 of these models, and then resort them by their primary condition.
        reduced_sorted_by_f1 = best_initial_sorted_by_f1.head(nr_candidates//2)
        best_secondary_sorted_by_conditions = self.sort_df(reduced_sorted_by_f1, optimize_metric)
        # At this point we should have filtered out bad outlier models, and be left with good candidates. 
        # We now select the best model according to the sort condidtions.
        best_model = best_secondary_sorted_by_conditions.head(1)

        return best_model

    
    def add_f1_stats(self, df_f1):
        df_f1.columns=df_f1.columns.str.strip()
        all_train_precision = df_f1['train_precision']
        all_train_recall = df_f1['train_recall']
        all_val_precision = df_f1['val_precision']
        all_val_recall = df_f1['val_recall']
        f1_train = self.create_f1_list(all_train_precision, all_train_recall)
        f1_val = self.create_f1_list(all_val_precision, all_val_recall)
        df_f1['train_f1'] = f1_train
        df_f1['val_f1'] = f1_val
        return df_f1

    

    def f1_score(self, precision, recall):
        f1 = 2*((precision*recall)/(precision + recall))
        return f1

    def create_f1_list(self, precision_df, recall_df):
        f1 = []
        for i in range(len(precision_df)):
            f1.append(self.f1_score(precision_df.loc[i], recall_df.loc[i]))
        return f1

        
    def sort_df(self, df, sort_conditions):
        ascending = False
        if sort_conditions == ['val_loss', 'train_loss'] or sort_conditions == ['train_loss', 'val_loss']:
            ascending = True
        if 'val_loss' in sort_conditions and 'train_loss' not in sort_conditions:
            raise Exception("Problematic sorting criteria. Cannot determine if sorting should be ascending or descending. A solution for this needs to be implemented in order for this to work")
        return df.sort_values(by=sort_conditions, axis = 0, ascending = ascending)

    """
    def convert_best_model_to_main_grid(self, best_model):
        model_dict = self.row_to_dict(best_model)


    def row_to_dict(self, model_df):
        keys = list(model_df.keys())
        # Assumes 10 columns dedicated to results and the rest to hyperparams
        hyper_keys = keys[:len(keys) - 10]
        model_dict = model_df[:len(hyper_keys)].to_dict()
        #del model_dict['index']
        return model_dict
    """

    def delete_metrics(self, best_model_df):
        best_model_df = best_model_df[best_model_df.columns[:len(best_model_df.columns) - 10]]
        return best_model_df

    def adapt_best_model_dict(self, best_model_dict):
        print(best_model_dict)
        return {key:[value] for (key,value) in best_model_dict.items()}

    def create_search_grid(self, main_model_grid):
        # Handle hyperparameters that are the same for all models
        param_grid = main_model_grid.copy()
        scaler = range(-4, 4, 2)
    
    
    def create_batch_params(self, batch_center):
        max_batch_size = 4096
        new_params = [batch_center//4, batch_center//2, batch_center*2, batch_center*4]
        for i, batch_size in enumerate(new_params):
            new_params[i] = min(batch_size, max_batch_size)
        return set(new_params)
    
    def create_learning_rate_params(self, learning_rate_center):
        min_learning_rate = 0.00001
        new_learning_params = [learning_rate_center*10**2, learning_rate_center*10**1, (learning_rate_center*10)/2, learning_rate_center / 2, learning_rate_center*10**(-1), learning_rate_center*10**(-2)]
        for i, rate in enumerate(new_learning_params):
            new_learning_params[i] = max(rate, min_learning_rate)
        return set(new_learning_params)

    def create_epochs_params(self, epoch_center):
        max_epochs = 150
        new_epochs = [epoch_center - 20, epoch_center -10, epoch_center + 10, epoch_center +20]
        for i in range(len(new_epochs)):
            new_epochs[i] = min(max(new_epochs[i], 10), max_epochs)
        return set(new_epochs)
    
    def create_optimizer_params(self, current_optimizer):
        options = ["adam", "rmsprop", "sgd"]
        return options[options != current_optimizer]

    def create_activation_params(self, current_activation, include_linear):
        if include_linear:
            options = ["linear", "relu", "softmax", "tanh", "sigmoid"]
        else: 
            options = ["relu", "softmax", "tanh", "sigmoid"]
        return options[options != current_activation]

    def create_reg_params(self, current_reg):
        max_reg = 0.3
        new_reg = [current_reg*10**2, current_reg*10, (current_reg*10)/2, current_reg/2, current_reg*10**(-1), current_reg*10**(-2)]
        for i in range(len(new_reg)):
            new_reg[i] = min(new_reg[i], max_reg)
        return set(new_reg)

    def create_boolean_params(self, current_bool):
        if current_bool:
            return [False, False]
        else:
            return [True, True]

class IncepTimeNarrowOptimizer(NarrowOptimizer):

    def __init__(self):
        super.__init__()

    def create_search_grid(self, main_model_dict):
        # Create grid that is to be serched
        pass

    def create_nr_modules_modules(self, current_nr_modules):
        max_modules = 30
        new_nr_modules = [center - 6, center - 3, center + 3, center + 6]
        for i in range(len(new_nr_modules)):
            new_nr_modules[i] = min(max(new_nr_modules[i], 1), max_modules)
        return set(new_nr_modules)
    
    def create_kernel_size(self, current_size):
        max_size = 100
        new_kernels = [current - 4, current -2, current +2, current +4]
        for i, kern in enumerate(new_kernels):
            new_kernels[i] = min(max(kern, 2), max_size)
        return set(new_kernels)
    
    


        

SyntaxError: invalid syntax (<ipython-input-16-4ab252674ad4>, line 204)

In [10]:
result_file_name = 'results_InceptionTime_NARROW_noiseNotNoise_detrend_timeAug_sscale_noiseAug_earlyS_highpass-0.1.csv'

narrowOpt = NarrowOptimizer(0, quick_mode = True, continue_from_result_file = True)
#top_10 = narrowOpt.get_best_model(result_file_name, 2, optimize_metric = ['val_accuracy', 'val_f1'], nr_candidates = 10)
best_model_dict = narrowOpt.run(result_file_name, 2, ['val_accuracy', 'val_f1'], 10)

Quick mode, starting of result file: results_InceptionTime_NARROW_noiseNotNoise_detrend_timeAug_sscale_noiseAug_earlyS_highpass-0.1.csv
{'batch_size': 128, 'epochs': 100, 'learning_rate': 0.001, 'optimizer': 'adam', 'bottleneck_size': 28, 'kernel_size': 60, 'l1_r': 0.0, 'l2_r': 0.0001, 'module_activation': 'tanh', 'module_output_activation': 'sigmoid', 'nr_modules': 23, 'num_filters': 38, 'output_activation': 'sigmoid', 'reg_module': True, 'reg_shortcut': False, 'shortcut_activation': 'relu', 'use_bottleneck': True, 'use_residuals': True}


In [11]:
pp = pprint.PrettyPrinter(indent=4)
pp.pprint(best_model_dict)

{   'batch_size': [128],
    'bottleneck_size': [28],
    'epochs': [100],
    'kernel_size': [60],
    'l1_r': [0.0],
    'l2_r': [0.0001],
    'learning_rate': [0.001],
    'module_activation': ['tanh'],
    'module_output_activation': ['sigmoid'],
    'nr_modules': [23],
    'num_filters': [38],
    'optimizer': ['adam'],
    'output_activation': ['sigmoid'],
    'reg_module': [True],
    'reg_shortcut': [False],
    'shortcut_activation': ['relu'],
    'use_bottleneck': [True],
    'use_residuals': [True]}


In [22]:
def dab(center):
    max_modules = 30
    new_nr_modules = [center - 6, center - 3, center + 3, center + 6]
    for i in range(len(new_nr_modules)):
        new_nr_modules[i] = min(new_nr_modules[i], max_modules)
    return set(new_nr_modules)
dab(27)

{21, 24, 30}